# 문장분류를 위한 CNN모델
'문장'을 '감정 클래스별'로 분류하는CNN 모델 구현
- CNN 은 이미지 분류 외에도 자연어 분류에도 좋은 성능을 냅니다 (단 **임베딩 품질**이 좋아야 함.)
- 이미지 이든, 자연어이든  수치(벡터) 로 표현 가능한 대상이면, 특징을 뽑아내도록 CNN 모델 학습인 가능한겁니다

In [1]:
# Hugging Face의 AutoTokenizer로 텍스트를 토큰화한 뒤, 간단한 TextCNN(컨볼루션 기반 문장 분류기)을
# PyTorch로 학습시키는 파이프라인을 만들어 볼겁니다..

# 주요 단계: 데이터 로드 → 토큰화/패딩 → Dataset / DataLoader → 모델 정의 → 학습 루프 → 평가 및 예측.

# import

In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from transformers import AutoTokenizer

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from datetime import datetime

# ---------------------------------------------------------------------
# 1. 설정 및 시드 고정
# ---------------------------------------------------------------------
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

generator = torch.Generator().manual_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ using PyTorch version: {torch.__version__}, Device: {device}")


✅ using PyTorch version: 2.8.0+cu126, Device: cuda


# 데이터 읽어오기

In [3]:
base_path = r'/content/drive/MyDrive/KoreaIT (코리아it)/250715 💚자연어처리S4/[자연어처리]/dataset(NLP2507)/chatbot_data'

In [4]:
data = pd.read_csv(os.path.join(base_path, 'chatbot_data.csv'))
data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


- 데이터셋 구조
    - Q (질문),  
    - A (답변)
    - label (감정)
        - **0**: 일상다반사
        - **1** : 이별(부정)
        - **2** : 사랑(긍정)

In [5]:
data['label'].unique()

array([0, 1, 2])

In [6]:
data['label'].value_counts()

,count
label,
0,5290
1,3570
2,2963


In [7]:
# texts : 모델 입력
# labels : 정수 레이블 (0, 1, 2)

texts = data['Q'].astype(str).tolist()
labels = data['label'].astype(int).tolist()

class_names = ['일상', '부정적', '긍정적']

In [8]:
texts[0], labels[0], class_names[labels[0]]

('12시 땡!', 0, '일상')

# 단어 시퀀스 - Hugging Face Tokenizer 사용

In [9]:
# 예: 한국어 모델 사용 (koBERT, KLUE-BERT 등)
#  처음 사용할 때는 모델/토크나이저 다운로드 발생
MODEL_NAME = "bert-base-multilingual-cased"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [10]:
tokenizer.encode('오늘은 맑은 아침이야')

[101, 9580, 118762, 10892, 9252, 10892, 9519, 119285, 10739, 21711, 102]

In [11]:
tokenizer.encode('오늘은 맑은 아침이야', truncation=True, max_length=7)

[101, 9580, 118762, 10892, 9252, 10892, 102]

In [12]:
tokenizer.encode('오늘은 맑은 아침이야', truncation=False, max_length=7)

[101, 9580, 118762, 10892, 9252, 10892, 9519, 119285, 10739, 21711, 102]

In [13]:
# max_length= 값이 없고 truncation=True 인 경우
# 토크나이저는 해당 토크나이저가 로딩한 사전학습 모델이 허용하는 최대 입력 길이를 기준으로 시퀀스를 잘라냅니다.
# 가령 BERT 와 같은 모델은 model_max_length 가 512 로 명확히 정의되어 있슴.
tokenizer.encode('오늘은 맑은 아침이야', truncation=True)

[101, 9580, 118762, 10892, 9252, 10892, 9519, 119285, 10739, 21711, 102]

In [14]:
# 길이 통계 기반으로 적절한 MAX_SEQ_LEN 설정
token_lenths = [len(tokenizer.encode(t, truncation=True)) for t in texts]
print(token_lenths)

[6, 11, 12, 13, 7, 9, 7, 8, 16, 15, 16, 7, 10, 10, 9, 7, 9, 10, 13, 11, 14, 9, 9, 10, 8, 11, 6, 9, 11, 8, 9, 11, 10, 11, 9, 9, 11, 12, 7, 11, 12, 8, 8, 8, 6, 10, 8, 9, 7, 11, 10, 6, 9, 8, 8, 7, 8, 8, 11, 15, 12, 10, 12, 14, 8, 13, 14, 13, 10, 16, 10, 8, 10, 11, 11, 11, 10, 13, 13, 13, 9, 12, 5, 7, 7, 10, 7, 8, 7, 5, 10, 11, 10, 5, 5, 9, 7, 13, 10, 11, 6, 7, 13, 9, 12, 8, 11, 6, 10, 8, 9, 6, 10, 11, 9, 19, 8, 9, 5, 8, 6, 9, 13, 5, 9, 10, 9, 12, 8, 6, 10, 6, 9, 9, 9, 12, 7, 9, 12, 11, 9, 16, 13, 13, 10, 7, 6, 9, 10, 9, 16, 10, 17, 11, 14, 10, 12, 8, 8, 10, 9, 11, 6, 8, 8, 7, 10, 9, 10, 11, 6, 10, 10, 8, 8, 8, 13, 12, 11, 14, 11, 8, 8, 10, 6, 10, 13, 5, 8, 11, 10, 8, 11, 11, 11, 10, 10, 13, 7, 6, 7, 8, 7, 9, 9, 7, 9, 8, 10, 12, 12, 8, 9, 8, 13, 9, 10, 9, 9, 11, 10, 9, 7, 8, 8, 8, 12, 11, 13, 7, 8, 9, 8, 10, 8, 8, 7, 11, 7, 7, 10, 11, 10, 7, 9, 8, 14, 10, 9, 7, 5, 11, 8, 9, 9, 8, 9, 10, 10, 10, 9, 7, 7, 8, 7, 14, 9, 8, 9, 11, 7, 12, 8, 7, 15, 11, 14, 11, 12, 10, 10, 9, 8, 9, 8, 10, 15, 10,

In [15]:
# 전체 문장 길이 분포의 95 퍼센타일로 정해 짧은 문장에도 과도하지 않게, 긴문장은 자를거다
MAX_SEQ_LEN = int(np.percentile(token_lenths, 95)) #95% 지점의 문장 토큰 길이
print(f'🟦 MAX_SEQ_LEN = {MAX_SEQ_LEN}')

🟦 MAX_SEQ_LEN = 19


In [16]:
# 모든 텍스트를 정수 시퀀스로 변환

encodings = tokenizer(
    texts,
    padding='max_length',
    truncation=True,
    max_length=MAX_SEQ_LEN,
    return_tensors='pt')

encodings

{'input_ids': tensor([[   101,  10186,  14040,  ...,      0,      0,      0],
        [   101,    122,  12508,  ...,      0,      0,      0],
        [   101,    124, 118963,  ...,      0,      0,      0],
        ...,
        [   101,  10016,  12310,  ...,      0,      0,      0],
        [   101,  10028,  90537,  ...,    136,    102,      0],
        [   101,  10028,  93200,  ...,      0,      0,      0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [17]:
# 특정 문장 확인
idx = 10
print(texts[idx])
print(encodings['input_ids'][idx])
tokenizer.convert_ids_to_tokens(encodings['input_ids'][idx])

SNS보면 나만 빼고 다 행복해보여
tensor([  101, 62342, 10731, 30005, 14867,  8982, 19105,  9391, 11664,  9056,
         9966, 70915, 14523, 30005, 29935,   102,     0,     0,     0])


['[CLS]',
 'SN',
 '##S',
 '##보',
 '##면',
 '나',
 '##만',
 '빼',
 '##고',
 '다',
 '행',
 '##복',
 '##해',
 '##보',
 '##여',
 '[SEP]',
 '[PAD]',
 '[PAD]',
 '[PAD]']

In [18]:
# 초반 n 개의 입력 문장과 변환된 encoding 출력해보기
n = 20
for t, e, _ in zip(texts, encodings['input_ids'], range(n)):
  print(f"{t} : {e.tolist()}")  # e 는 PyTorch Tensor다


12시 땡! : [101, 10186, 14040, 100, 106, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1지망 학교 떨어졌어 : [101, 122, 12508, 89292, 9953, 25242, 9141, 12965, 119210, 12965, 102, 0, 0, 0, 0, 0, 0, 0, 0]
3박4일 놀러가고 싶다 : [101, 124, 118963, 11011, 18392, 9025, 30873, 11287, 11664, 9495, 11903, 102, 0, 0, 0, 0, 0, 0, 0]
3박4일 정도 놀러가고 싶다 : [101, 124, 118963, 11011, 18392, 107657, 9025, 30873, 11287, 11664, 9495, 11903, 102, 0, 0, 0, 0, 0, 0]
PPL 심하네 : [101, 33428, 11369, 9491, 35506, 77884, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SD카드 망가졌어 : [101, 27589, 24206, 15001, 9255, 11287, 119210, 12965, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SD카드 안돼 : [101, 27589, 24206, 15001, 9521, 118798, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SNS 맞팔 왜 안하지ㅠㅠ : [101, 62342, 10731, 9256, 119378, 9596, 100, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SNS 시간낭비인 거 아는데 매일 하는 중 : [101, 62342, 10731, 9485, 18784, 118719, 29455, 12030, 8863, 9519, 41850, 9258, 18392, 23969, 9694, 102, 0, 0, 0]
SNS 시간낭비인데 자꾸 보게됨 : [101, 62342, 10731, 9485, 1

In [19]:
# 어휘 사전 크기, -> Embedding 레이어의 입력 크기로 사용될거다
VOCAB_SIZE = tokenizer.vocab_size
print(f'VOCAB_SIZE = {VOCAB_SIZE}')

VOCAB_SIZE = 119547


## pad token id 설정

In [20]:
tokenizer.pad_token_id

# 어떤 토크나이저에서 pad_token_id가 0이 아닐 수 있으니,
# nn.Embedding(..., padding_idx=tokenizer.pad_token_id)로 설정하는 것이 안전합니다.


0

In [21]:
pad_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else 0
pad_id

0

# DataSet, DataLoader 준비

In [22]:
encodings.items()

ItemsView({'input_ids': tensor([[   101,  10186,  14040,  ...,      0,      0,      0],
        [   101,    122,  12508,  ...,      0,      0,      0],
        [   101,    124, 118963,  ...,      0,      0,      0],
        ...,
        [   101,  10016,  12310,  ...,      0,      0,      0],
        [   101,  10028,  90537,  ...,    136,    102,      0],
        [   101,  10028,  93200,  ...,      0,      0,      0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 0, 0, 0]])})

In [23]:
X_temp = {k: v.numpy() for k, v in encodings.items()}
X = X_temp['input_ids']

print(X.shape)
X  # X 는 (num_samples, MAX_SEQ_LEN) 형태의 정수 배열

(11823, 19)


array([[   101,  10186,  14040, ...,      0,      0,      0],
       [   101,    122,  12508, ...,      0,      0,      0],
       [   101,    124, 118963, ...,      0,      0,      0],
       ...,
       [   101,  10016,  12310, ...,      0,      0,      0],
       [   101,  10028,  90537, ...,    136,    102,      0],
       [   101,  10028,  93200, ...,      0,      0,      0]])

In [24]:
# train:test = 9:1
X_train, X_test, y_train, y_test = train_test_split(
    X, labels, test_size=0.1, random_state=SEED, stratify=labels
)

# train 을 다시 분리 => train:val = 8:2
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=SEED, stratify=y_train
)

print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")


Train: 8512, Val: 2128, Test: 1183


In [25]:
# Dataset 클래스 정의

class EmotionDataset(Dataset):
  def __init__(self, X, y):
    self.X = torch.tensor(X, dtype=torch.long)
    self.y = torch.tensor(y, dtype=torch.long)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]

train_ds = EmotionDataset(X_train, y_train)
val_ds = EmotionDataset(X_val, y_val)
test_ds = EmotionDataset(X_test, y_test)

In [26]:
# DataLoader 준비
BATCH_SIZE = 32

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, generator=generator)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, generator=generator)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, generator=generator)

# 각 배치의 개수
print(f"Train: {len(train_loader)}, Val: {len(val_loader)}, Test: {len(test_loader)}")

Train: 266, Val: 67, Test: 37


# TextCNN 모델 정의

In [27]:
# Conv2d => Conv1d

In [28]:
# TextCNN 모델 정의
#   핵심 아이디어: 서로 다른 크기의 n-gram(3,4,5)을 잡아내는 필터들을 병렬로 적용하고,
#                 각 필터별로 전역 풀링해 고정 길이 피처로 합침.

In [29]:
class TextCNN(nn.Module):
  def __init__(self, vocab_size, embed_dim, num_classes, dropout=0.5, kernel_sizes=[3,4,5], num_filters=128):
    super(TextCNN, self).__init__()

    self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embed_dim,
                                  padding_idx=pad_id)

    # 입력: input_ids 의 shape: [batch_size, seq_len]
    # 출력: [batch_size, seq_len, embed_dim]

    self.convs = nn.ModuleList([
      nn.Sequential(
          nn.Conv1d(in_channels=embed_dim, out_channels=num_filters, kernel_size=k),
          # 출력 shape : [batch_size, num_filters, seq_len - k + 1]
          nn.ReLU(),

          nn.BatchNorm1d(num_filters),

          nn.MaxPool1d(kernel_size=(MAX_SEQ_LEN -k + 1))
          # 출력 [batch_size, num_filters, 1]
      )
      for k in kernel_sizes
    ])

    self.dropout = nn.Dropout(dropout)
    self.fc = nn.Linear(num_filters * len(kernel_sizes), num_classes)

  def forward(self, x):
    x = self.embedding(x)
    # -> [batch, seq_len, embed_dim]

    x = x.permute(0, 2, 1)
    # -> [batch, embed_dim, seq_len]
    #   Conv1d 는 채널차원이 먼저 오도록 되어 있다.

    conv_outs = [conv(x).squeeze(2) for conv in self.convs  ]
    # 각 kernel_size 마다
    # [batch_size, num_filters, 1] -> .squeeze(2)
    #   ==> [batch_size, num_filters]

    out = torch.cat(conv_outs, dim=1)
    # 모든 커널 결과를 torch.cat 하면 -> [batch_size, num_filters * len(kernel_sizes)]

    out = self.dropout(out)

    return self.fc(out)
    # 출력: [batch_size, num_classes]


NUM_CLASSES = len(set(labels))
model = TextCNN(vocab_size=VOCAB_SIZE, embed_dim=128, num_classes=NUM_CLASSES).to(device)





In [30]:
model_total = 0
for name, child in model.named_children():
  print('🔶', name)
  layer_total = 0
  for param_name, param in child.named_parameters():
    print('  🟡', f"{param_name:9s}", ':', f"{param.numel():,}개", '←', param.data.shape)
    layer_total += param.numel()
  if layer_total:
    print('  [layer params]:', f"{layer_total:,}개") # parameter 있는 레이어만 출력
    model_total += layer_total
  print()

print(f'[model params]:', f"{model_total:,}개")

🔶 embedding
  🟡 weight    : 15,302,016개 ← torch.Size([119547, 128])
  [layer params]: 15,302,016개

🔶 convs
  🟡 0.0.weight : 49,152개 ← torch.Size([128, 128, 3])
  🟡 0.0.bias  : 128개 ← torch.Size([128])
  🟡 0.2.weight : 128개 ← torch.Size([128])
  🟡 0.2.bias  : 128개 ← torch.Size([128])
  🟡 1.0.weight : 65,536개 ← torch.Size([128, 128, 4])
  🟡 1.0.bias  : 128개 ← torch.Size([128])
  🟡 1.2.weight : 128개 ← torch.Size([128])
  🟡 1.2.bias  : 128개 ← torch.Size([128])
  🟡 2.0.weight : 81,920개 ← torch.Size([128, 128, 5])
  🟡 2.0.bias  : 128개 ← torch.Size([128])
  🟡 2.2.weight : 128개 ← torch.Size([128])
  🟡 2.2.bias  : 128개 ← torch.Size([128])
  [layer params]: 197,760개

🔶 dropout

🔶 fc
  🟡 weight    : 1,152개 ← torch.Size([3, 384])
  🟡 bias      : 3개 ← torch.Size([3])
  [layer params]: 1,155개

[model params]: 15,500,931개


# 손실함수, optimizer

In [31]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# 학습함수, 검증함수

In [32]:
def evaluate(model, loader):
  model.eval()

  correct, total = 0, 0

  with torch.no_grad():
    for X_batch, y_batch in loader:
      X_batch, y_batch = X_batch.to(device), y_batch.to(device)
      outputs = model(X_batch)
      _, preds = torch.max(outputs, 1)
      correct += (preds == y_batch).sum().item()
      total += y_batch.size(0)

  return correct / total

def train_model(model, train_loader, val_loader, epochs=10):

  best_val_acc = 0  # 가장 성능이 좋은 모델의 val acc 기록

  for epoch in range(1, epochs + 1):

    model.train()

    total_loss, correct, total = 0, 0, 0

    # 매 배치마다 loss 계산 -> 역전파 -> 파라미터 업데이트
    for X_batch, y_batch in train_loader:
      X_batch, y_batch = X_batch.to(device), y_batch.to(device)

      optimizer.zero_grad()
      outputs = model(X_batch)
      loss = criterion(outputs, y_batch)
      loss.backward()
      optimizer.step()

      total_loss += loss.item()
      _, preds = torch.max(outputs, 1)
      correct += (preds == y_batch).sum().item()
      total += y_batch.size(0)

    # epoch 끝마다 검증 성능 (evaluate) 계산
    train_acc = correct / total
    val_acc = evaluate(model, val_loader)
    print(f"[Epoch {epoch:02d}] Train Loss: {total_loss/len(train_loader):.4f} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")


    # 검증 정확도가 최고일때 모델 저장.
    if val_acc > best_val_acc:
      best_val_acc = val_acc
      torch.save(model.state_dict(), 'best_textcnn.pt')
      print('✅ Best model saved: val_acc=', best_val_acc)


# 모델학습

In [33]:
EPOCHS = 15
train_model(model, train_loader, val_loader, epochs=EPOCHS)

[Epoch 01] Train Loss: 0.9388 | Train Acc: 0.6535 | Val Acc: 0.7768
✅ Best model saved: val_acc= 0.7767857142857143
[Epoch 02] Train Loss: 0.5946 | Train Acc: 0.7531 | Val Acc: 0.8073
✅ Best model saved: val_acc= 0.8073308270676691
[Epoch 03] Train Loss: 0.5030 | Train Acc: 0.8016 | Val Acc: 0.8125
✅ Best model saved: val_acc= 0.8125
[Epoch 04] Train Loss: 0.4207 | Train Acc: 0.8376 | Val Acc: 0.8158
✅ Best model saved: val_acc= 0.8157894736842105
[Epoch 05] Train Loss: 0.3870 | Train Acc: 0.8537 | Val Acc: 0.8078
[Epoch 06] Train Loss: 0.3238 | Train Acc: 0.8796 | Val Acc: 0.8261
✅ Best model saved: val_acc= 0.8261278195488722
[Epoch 07] Train Loss: 0.2922 | Train Acc: 0.8920 | Val Acc: 0.8163
[Epoch 08] Train Loss: 0.2598 | Train Acc: 0.9011 | Val Acc: 0.8308
✅ Best model saved: val_acc= 0.8308270676691729
[Epoch 09] Train Loss: 0.2396 | Train Acc: 0.9134 | Val Acc: 0.8318
✅ Best model saved: val_acc= 0.831766917293233
[Epoch 10] Train Loss: 0.2104 | Train Acc: 0.9249 | Val Acc: 0.81

# 테스트 평가

In [34]:
model.load_state_dict(torch.load('best_textcnn.pt'))

<All keys matched successfully>

In [35]:
model.eval()

y_true, y_pred = [], []
with torch.no_grad():
  for X_batch, y_batch in test_loader:
    X_batch = X_batch.to(device)
    outputs = model(X_batch)
    preds = torch.argmax(outputs, dim=1).cpu().numpy()
    y_true.extend(y_batch.numpy())
    y_pred.extend(preds)

print('\n✅ Test Classification Report')
print(classification_report(y_true, y_pred, digits=4))



✅ Test Classification Report
              precision    recall  f1-score   support

           0     0.8623    0.8166    0.8388       529
           1     0.8005    0.8207    0.8105       357
           2     0.8070    0.8586    0.8320       297

    accuracy                         0.8284      1183
   macro avg     0.8233    0.8320    0.8271      1183
weighted avg     0.8298    0.8284    0.8286      1183



# 예측

In [36]:
# 예측 함수
# 토큰화 → 모델 통과 → softmax로 확률 추출 → argmax로 예측 클래스 도출.
# 출력: 각 클래스별 확률과 예측 결과.

def predict_emotion(text):
  encoded = tokenizer(text,
            padding='max_length',
            truncation=True,
            max_length=MAX_SEQ_LEN,
            return_tensors='pt')

  with torch.no_grad():
    probs = torch.softmax(model(encoded['input_ids'].to(device)), dim=1)[0].cpu().numpy()
    pred_idx = np.argmax(probs)


  print(f'🟦 입력: {text}')
  for cls, p in zip(class_names, probs):  # 각 클래스별 예측 확률
    print(f'   {cls:<5}: {p:.3f}')

  # 예측결과
  print(f'👉 예측 결과: {class_names[pred_idx]}\n')



# 실제 문장 테스트

In [37]:
sentences = [
    "가스불 켜놓고 나온거 같아",
    "어떻게 해야 여자친구가 제 진심을 알아줄까",
    "업무 스트레스 넘 심해",
    "남친이랑 뭐하고 놀지",
    "내가 좋아하는 사람이 날 좋아했으면 좋겠어.",
    "원본 데이터가 이상해요",
    "내가 최대한 잘해줘도 불만이 많아",
]

for s in sentences:
    predict_emotion(s)

🟦 입력: 가스불 켜놓고 나온거 같아
   일상   : 1.000
   부정적  : 0.000
   긍정적  : 0.000
👉 예측 결과: 일상

🟦 입력: 어떻게 해야 여자친구가 제 진심을 알아줄까
   일상   : 0.000
   부정적  : 0.992
   긍정적  : 0.008
👉 예측 결과: 부정적

🟦 입력: 업무 스트레스 넘 심해
   일상   : 0.385
   부정적  : 0.551
   긍정적  : 0.064
👉 예측 결과: 부정적

🟦 입력: 남친이랑 뭐하고 놀지
   일상   : 0.018
   부정적  : 0.000
   긍정적  : 0.982
👉 예측 결과: 긍정적

🟦 입력: 내가 좋아하는 사람이 날 좋아했으면 좋겠어.
   일상   : 0.001
   부정적  : 0.000
   긍정적  : 0.999
👉 예측 결과: 긍정적

🟦 입력: 원본 데이터가 이상해요
   일상   : 0.321
   부정적  : 0.048
   긍정적  : 0.631
👉 예측 결과: 긍정적

🟦 입력: 내가 최대한 잘해줘도 불만이 많아
   일상   : 0.039
   부정적  : 0.000
   긍정적  : 0.961
👉 예측 결과: 긍정적

